<a href="https://colab.research.google.com/github/boscoybarra/customer-transaction-prediction/blob/master/customer_transaction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading Kaggle Dataset and dependencies


In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

 Budgets
'Colab Notebooks'
'Contact Clinics in Spain.gsheet'
 Facturas
 kaggle
'Manual instalación Recall en clínica.gdoc'
'RECORDATORIOS: MENSAJES DE TEXTO PARA PACIENTES Shared.gslides'
'RECORDATORIOS: MENSAJES DE TEXTO PARA PACIENTES Shared SMALL.gslides'


In [0]:
# Install kaggle pip
!pip install -U -q kaggle

!mkdir ~/.kaggle/

# We copy the json file into a apparently temp file .kaggle
!cp "/content/drive/My Drive/kaggle/kaggle.json" ~/.kaggle/




In [0]:
!chmod 600 /root/.kaggle/kaggle.json

##Download the data 

In [5]:
!kaggle competitions download santander-customer-transaction-prediction
!ls



 97% 119M/122M [00:00<00:00, 143MB/s] 
100% 122M/122M [00:00<00:00, 157MB/s]
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 137MB/s]
 91% 111M/122M [00:00<00:00, 155MB/s] 
100% 122M/122M [00:00<00:00, 132MB/s]
drive  sample_data  sample_submission.csv.zip  test.csv.zip  train.csv.zip


#Starting point


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics

import matplotlib.pyplot as plt
from scipy import stats

In [0]:
train = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
test = pd.read_csv('test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
sample_submission = pd.read_csv('sample_submission.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
# targets = train[train.columns[1:2]]
# train = train[train.columns[2:]]
# train = train[train.columns[2:]]
# test = test[test.columns[1:]]


In [0]:
target_transactions = targets.sum(axis=None)
print(target_transactions)

targets.shape

target    20098
dtype: int64


(200000, 1)

In [12]:
!pip install h2o
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

    100% |████████████████████████████████| 120.9MB 259kB/s 
  Stored in directory: /root/.cache/pip/wheels/76/73/b8/8639930fbc56e9f26fac210f5e65dbe05a0c39396ddaf4a685
Successfully built h2o


NameError: ignored

In [14]:
! apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.10-63ubuntu1~02).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
openjdk version "10.0.2" 2018-07-17
OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4)
OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)


In [16]:
! pip install h2o

In [0]:
import h2o

In [29]:
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqfiziffi
  JVM stdout: /tmp/tmpqfiziffi/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpqfiziffi/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_unknownUser_eufg7n
H2O cluster total nodes:,1
H2O cluster free memory:,16 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [30]:
train = h2o.import_file(train)

H2OTypeError: ignored

In [31]:
test = h2o.import_file(test)

H2OTypeError: ignored

In [20]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [21]:
train.shape

(200000, 202)

In [22]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [23]:
test.shape

(200000, 201)

In [0]:
x = test.columns[1:]
y = 'target'

In [25]:
x

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

In [26]:
y

'target'

In [32]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

AttributeError: ignored

In [28]:
aml = H2OAutoML(max_models=180, seed=47, max_runtime_secs=31000)
aml.train(x=x, y=y, training_frame=train)

H2OConnectionError: ignored

In [33]:
aml = H2OAutoML(max_models=180, seed=47, max_runtime_secs=31000)
aml.train(x=x, y=y, training_frame=train)

AttributeError: ignored

In [34]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
This H2OFrame is empty.


In [35]:
preds = aml.predict(test)

TypeError: ignored

In [36]:
preds['p1'].as_data_frame().values.flatten().shape

NameError: ignored

In [37]:
preds

NameError: ignored

In [38]:
  sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission.shape

FileNotFoundError: ignored

In [39]:
sample_submission['target'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('submission.csv', index=False)

NameError: ignored

In [0]:
x = targets[1:]
y = train[:1]

print(y)
print(x)



    var_0   var_1    var_2  var_3    var_4   var_5   var_6    var_7  var_8  \
0  8.9255 -6.7863  11.9081  5.093  11.4607 -9.2834  5.1187  18.6266  -4.92   

   var_9   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  5.747   ...      4.4354   3.9642   3.1364    1.691  18.5227  -2.3978   

   var_196  var_197  var_198  var_199  
0   7.8784   8.5635  12.7803  -1.0914  

[1 rows x 200 columns]
        target
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0
10           0
11           0
12           0
13           1
14           0
15           0
16           0
17           0
18           0
19           0
20           0
21           0
22           0
23           0
24           0
25           0
26           0
27           0
28           0
29           1
30           0
...        ...
199970       0
199971       0
199972       0
199973       0
199974       0
199975       0
199976       1
199

##Plot what we have

In [0]:
# np.random.seed(12345678)
# x = np.random.random(10)
# y = 1.6*x + np.random.random(10)

slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print("slope: %f    intercept: %f" % (slope, intercept))

ValueError: ignored

In [0]:
print("r-squared: %f" % r_value**2)

NameError: ignored

In [0]:
plt.plot(x, y, 'o', label='original data')
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.legend()
plt.show()

NameError: ignored

#### TODO 0: get all values that have a target of 1 and try to generate a regression for all 200 variables plus passing the other 80% of data set that has ceros.


#### TODO 1: train with 20098 taregets vs another 20098 random 0 targets ?



In [0]:
features

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

In [0]:
train_test = pd.concat([train, test], axis =0)



target = train_test['target'].values

test_target = train['target']


In [0]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 17,
         "metric": 'auc',
         "verbosity": -1}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train_test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_test.values, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_test.iloc[trn_idx][features], label=target[trn_idx])
    val_data = lgb.Dataset(train_test.iloc[val_idx][features], label=target[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_test.iloc[val_idx][features], num_iteration=clf.best_iteration)

fold n°0
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.633934	valid_1's auc: 0.511805
[200]	training's auc: 0.680832	valid_1's auc: 0.512238
Early stopping, best iteration is:
[128]	training's auc: 0.649589	valid_1's auc: 0.512908
fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.635347	valid_1's auc: 0.511121
[200]	training's auc: 0.681405	valid_1's auc: 0.514333
[300]	training's auc: 0.713812	valid_1's auc: 0.515166


KeyboardInterrupt: ignored

Let's take a look at the overall oof CV AUC:

In [0]:
metrics.roc_auc_score(target, oof)

AUC of 0.52 is overall not a major issue, but it is slightly over 0.50, which can have an impact when you are chasing the 4th decimal point.

## Lasso Implementation 
https://www.kaggle.com/baogorek/lasso-regression-with-keras